In [1]:
# Import libraries
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# import tensorflow as tf
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# print(sess)

# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2025431843213789107
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6669678674
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2694520509188061999
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
import os
# os.listdir()
os.chdir('../resources/raw/ml-latest/ml-latest')


In [4]:
ratings = pd.read_csv('ratings.csv')

ratings.head()

userId  movieId  rating   timestamp
0       1      307     3.5  1256677221
1       1      481     3.5  1256677456
2       1     1091     1.5  1256677471
3       1     1257     4.5  1256677460
4       1     1449     4.5  1256677264

In [5]:
ratings['userEmbId'] = ratings['userId'] - 1
ratings['movieEmbId'] = ratings['movieId'] - 1
# Set max_userid to the maximum user_id in the ratings
max_userid = ratings['userId'].drop_duplicates().max()
# Set max_movieid to the maximum movie_id in the ratings
max_movieid = ratings['movieId'].drop_duplicates().max()

ratings.head()

userId  movieId  rating   timestamp  userEmbId  movieEmbId
0       1      307     3.5  1256677221          0         306
1       1      481     3.5  1256677456          0         480
2       1     1091     1.5  1256677471          0        1090
3       1     1257     4.5  1256677460          0        1256
4       1     1449     4.5  1256677264          0        1448

In [6]:
users = pd.read_csv('ratings.csv')

users = users.drop_duplicates(subset='userId')

users = users['userId'].reset_index()

users = users.drop(columns='index')

In [7]:
users.head()

userId
0       1
1       2
2       3
3       4
4       5

In [8]:
movies = pd.read_csv('movies.csv')

movies.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [ ]:
genres = pd.get_dummies(movies.genres.str.split('|',expand=True).stack()).sum(level=0)

In [ ]:
movies = pd.concat([movies, genres], axis=1)
movies = movies.drop(columns=['genres','(no genres listed)'])

In [ ]:
movies.head()

movieId                               title  Action  Adventure  Animation  \
0        1                    Toy Story (1995)       0          1          1   
1        2                      Jumanji (1995)       0          1          0   
2        3             Grumpier Old Men (1995)       0          0          0   
3        4            Waiting to Exhale (1995)       0          0          0   
4        5  Father of the Bride Part II (1995)       0          0          0   

   Children  Comedy  Crime  Documentary  Drama  ...  Film-Noir  Horror  IMAX  \
0         1       1      0            0      0  ...          0       0     0   
1         1       0      0            0      0  ...          0       0     0   
2         0       1      0            0      0  ...          0       0     0   
3         0       1      0            0      1  ...          0       0     0   
4         0       1      0            0      0  ...          0       0     0   

   Musical  Mystery  Romance  Sci-Fi  Thriller  War  Western  
0        0        0        0       0         0    0        0  
1        0        0        0       0         0    0        0  
2        0        0        1       0         0    0        0  
3        0        0        1       0         0    0        0  
4        0        0        0       0         0    0        0  

[5 rows x 21 columns]

In [ ]:
shuffled_ratings = ratings.sample(frac=1., random_state=42)

# Shuffling users
Users = shuffled_ratings['userEmbId'].values
print('Users:', Users, ', shape =', Users.shape)

# Shuffling movies
Movies = shuffled_ratings['movieEmbId'].values
print('Movies:', Movies, ', shape =', Movies.shape)

# Shuffling ratings
Ratings = shuffled_ratings['rating'].values
print('Ratings:', Ratings, ', shape =', Ratings.shape)

Users: [268199 127229 251067 ... 238529 164461 237214] , shape = (27753444,)
Movies: [2010 1526 1395 ...  589 4642 1171] , shape = (27753444,)
Ratings: [5.  3.5 3.5 ... 3.  3.5 3. ] , shape = (27753444,)


In [ ]:
import keras
from keras.layers import Input, Embedding, dot, Dense, Reshape
from keras.models import Model, Sequential

Using TensorFlow backend.


In [ ]:
user_input = Input(shape=(1,), name='user_input')
x = Embedding(input_dim=max_userid, output_dim=100, input_length=1)(user_input)
user_output = Reshape((100,))(x)

In [ ]:
movie_input = Input(shape=(1,), name='movie_input')
y = Embedding(input_dim=max_movieid, output_dim=100, input_length=1)(movie_input)
movie_output = Reshape((100,))(y)

In [ ]:
z = dot([Dense(10)(user_output), Dense(10)(movie_output)], axes=1)

In [ ]:
model = Model(inputs=[user_input, movie_input], outputs=z)

In [ ]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
movie_input (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 100)       28322800    user_input[0][0]                 
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 100)       19388600    movie_input[0][0]                
__________________________________________________________________________________________________
reshape_1 

In [ ]:
model.compile(loss='mse', optimizer='adamax')

In [ ]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

In [ ]:
callbacks = [EarlyStopping('val_loss', patience=2),
            ModelCheckpoint('weights.h5', save_best_only=True)]

history=model.fit([Users, Movies], Ratings, epochs=30, validation_split=.1, verbose=2, callbacks=callbacks)

Train on 24978099 samples, validate on 2775345 samples
Epoch 1/30


In [ ]:
# Show the best validation RMSE
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print('Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss)))